In [4]:
import os
import json
import pandas as pd

def coco_to_binary_csv(dataset_dir):
    subsets = ['train', 'valid', 'test']
    all_csvs = {}

    for subset in subsets:
        json_path = os.path.join(dataset_dir, subset, '_annotations.coco.json')
        with open(json_path, 'r') as f:
            data = json.load(f)

        # image_id → filename
        id_to_file = {img['id']: img['file_name'] for img in data['images']}

        # Соберем все category_id для каждого image_id
        image_labels = {}
        for ann in data['annotations']:
            img_id = ann['image_id']
            cat_id = ann['category_id']
            if img_id not in image_labels:
                image_labels[img_id] = []
            image_labels[img_id].append(cat_id)

        # Для каждого изображения определим итоговую метку
        rows = []
        for img_id, labels in image_labels.items():
            filename = id_to_file[img_id]

            # Если есть хотя бы один 'Effected' (category_id == 1) → label = 1
            if 1 in labels:
                label = 1
            else:
                label = 0  # всё нормальное
            rows.append((filename, label))

        # Преобразуем в DataFrame и сохраним
        df = pd.DataFrame(rows, columns=['filename', 'label'])
        csv_path = os.path.join(dataset_dir, f'{subset}_labels.csv')
        df.to_csv(csv_path, index=False)
        all_csvs[subset] = df

    return all_csvs

# Запуск
dataset_path = 'sinusitis'  # Заменить на свой путь
csvs = coco_to_binary_csv(dataset_path)


In [7]:
import os
import json
import pandas as pd

def coco_to_detailed_csv(dataset_dir):
    subsets = ['train', 'valid', 'test']
    all_csvs = {}

    for subset in subsets:
        json_path = os.path.join(dataset_dir, subset, '_annotations.coco.json')
        with open(json_path, 'r') as f:
            data = json.load(f)

        # image_id → filename
        id_to_file = {img['id']: img['file_name'] for img in data['images']}

        # Собираем категории аннотаций для каждого image_id
        image_labels = {}
        for ann in data['annotations']:
            img_id = ann['image_id']
            cat_id = ann['category_id']
            if img_id not in image_labels:
                image_labels[img_id] = []
            image_labels[img_id].append(cat_id)

        # Собираем итоговую таблицу
        rows = []
        for img_id, labels in image_labels.items():
            filename = id_to_file[img_id]

            # Гарантируем 2 аннотации (иначе дополним 2 как Normal)
            if len(labels) < 2:
                labels += [2] * (2 - len(labels))

            # Заменяем 2 на 0 (Normal → 0)
            label1 = 0 if labels[0] == 2 else labels[0]
            label2 = 0 if labels[1] == 2 else labels[1]

            # Результат: 1, если хотя бы один воспалённый синус
            result_label = 1 if 1 in [label1, label2] else 0
            rows.append((filename, label1, label2, result_label))

        # Создаём DataFrame и сохраняем
        df = pd.DataFrame(rows, columns=['filename', 'label1', 'label2', 'result_label'])
        csv_path = os.path.join(dataset_dir, f'{subset}_detailed_labels.csv')
        df.to_csv(csv_path, index=False)
        all_csvs[subset] = df

    return all_csvs

# Пример запуска
dataset_path = 'sinusitis'  # путь к датасету
csvs = coco_to_detailed_csv(dataset_path)


In [31]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import numpy as np
import random

# ✅ Reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# ✅ Настройки путей
csv_file = "sinusitis/full_labels.csv"
image_dir = "sinusitis/full_imgs"

# ✅ Кастомный Dataset
class SinusDataset(Dataset):
    def __init__(self, df, images_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.images_dir = images_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.images_dir, row['filename'])
        image = Image.open(img_path).convert("RGB")
        label = int(row['result_label'])
        if self.transform:
            image = self.transform(image)
        return image, label

# ✅ Трансформации
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# ✅ Модель
def create_model():
    model = resnet18(weights=ResNet18_Weights.DEFAULT)
    model.fc = nn.Linear(model.fc.in_features, 2)  # binary classification
    return model

# ✅ Обучение и валидация
def train_val(model, train_loader, val_loader, device, epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    model.to(device)

    for epoch in range(epochs):
        model.train()
        train_loss, train_correct = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            train_correct += (outputs.argmax(1) == labels).sum().item()

        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_correct += (outputs.argmax(1) == labels).sum().item()
                val_total += labels.size(0)

        print(f"Epoch {epoch+1}: Train acc = {train_correct/len(train_loader.dataset):.4f}, "
              f"Val acc = {val_correct/val_total:.4f}")

    return model

# ✅ Кросс-валидация
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

df = pd.read_csv(csv_file)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['result_label'])):
    print(f"\n=== Fold {fold+1} ===")
    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]

    train_dataset = SinusDataset(train_df, image_dir, transform=train_transform)
    val_dataset = SinusDataset(val_df, image_dir, transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    model = create_model()
    model = train_val(model, train_loader, val_loader, device, epochs=10)

    # Оценка модели после обучения
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            preds = outputs.argmax(1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    print(classification_report(all_labels, all_preds))
    torch.save(model.state_dict(), f"resnet18_fold{fold+1}.pth")




=== Fold 1 ===
Epoch 1: Train acc = 0.6481, Val acc = 0.6429
Epoch 2: Train acc = 0.8148, Val acc = 0.6786
Epoch 3: Train acc = 0.8611, Val acc = 0.7143
Epoch 4: Train acc = 0.9167, Val acc = 0.7143
Epoch 5: Train acc = 0.9444, Val acc = 0.8571
Epoch 6: Train acc = 0.9537, Val acc = 0.7143
Epoch 7: Train acc = 0.9907, Val acc = 0.7143
Epoch 8: Train acc = 0.9815, Val acc = 0.8929
Epoch 9: Train acc = 0.9907, Val acc = 0.8571
Epoch 10: Train acc = 0.9815, Val acc = 0.8214
              precision    recall  f1-score   support

           0       1.00      0.62      0.76        13
           1       0.75      1.00      0.86        15

    accuracy                           0.82        28
   macro avg       0.88      0.81      0.81        28
weighted avg       0.87      0.82      0.81        28


=== Fold 2 ===
Epoch 1: Train acc = 0.6330, Val acc = 0.8148
Epoch 2: Train acc = 0.8349, Val acc = 0.8148
Epoch 3: Train acc = 0.8899, Val acc = 0.7778
Epoch 4: Train acc = 0.9541, Val acc = 0.7

In [32]:
import os
import torch
import pandas as pd
import numpy as np
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from PIL import Image
from torch import nn, optim
from tqdm import tqdm

# Путь к папке и таблице
image_dir = 'sinusitis/full_imgs'
csv_path = 'sinusitis/full_labels.csv'

# Загружаем таблицу
df = pd.read_csv(csv_path)

# Фильтруем только строки с опухолью
df = df[df['result_label'] == 1].copy()

# Присваиваем класс: 0 - левая, 1 - правая, 2 - обе
def get_tumor_side(row):
    if row['label1'] == 1 and row['label2'] == 0:
        return 0
    elif row['label1'] == 0 and row['label2'] == 1:
        return 1
    else:
        return 2

df['class'] = df.apply(get_tumor_side, axis=1)

# Настраиваем трансформации
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Dataset
class TumorDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.df.loc[idx, 'filename'])
        image = Image.open(img_path).convert("RGB")
        label = self.df.loc[idx, 'class']
        if self.transform:
            image = self.transform(image)
        return image, label

# Модель
def get_model():
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 3)  # 3 класса
    return model

# Тренировка 1 фолда
def train_one_fold(model, train_loader, val_loader, criterion, optimizer, device, fold):
    best_acc = 0
    os.makedirs('models_tumor_side', exist_ok=True)

    for epoch in range(10):
        model.train()
        for images, labels in tqdm(train_loader, desc=f"Fold {fold} Epoch {epoch+1}/10 - Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = correct / total
        print(f"Fold {fold} Epoch {epoch+1}: Val Accuracy = {acc:.4f}")

        # Сохраняем модель, если она лучше предыдущей
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), f'models_tumor_side/best_model_fold{fold}.pt')

# Кросс-валидация
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['class'])):
    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]

    train_dataset = TumorDataset(train_df, image_dir, transform=transform)
    val_dataset = TumorDataset(val_df, image_dir, transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

    model = get_model().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    train_one_fold(model, train_loader, val_loader, criterion, optimizer, device, fold)


/Users/arnellatolegen/Desktop/sinus_ct_project/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/arnellatolegen/Desktop/sinus_ct_project/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Fold 0 Epoch 1/10 - Training: 100%|██████████| 4/4 [00:04<00:00,  1.05s/it]


Fold 0 Epoch 1: Val Accuracy = 0.6000


Fold 0 Epoch 2/10 - Training: 100%|██████████| 4/4 [00:00<00:00,  9.17it/s]


Fold 0 Epoch 2: Val Accuracy = 0.5333


Fold 0 Epoch 3/10 - Training: 100%|██████████| 4/4 [00:00<00:00,  9.62it/s]


...

Fold 4 Epoch 8: Val Accuracy = 0.6429


Fold 4 Epoch 9/10 - Training: 100%|██████████| 4/4 [00:00<00:00,  9.57it/s]


Fold 4 Epoch 9: Val Accuracy = 0.6429


Fold 4 Epoch 10/10 - Training: 100%|██████████| 4/4 [00:00<00:00,  9.62it/s]

Fold 4 Epoch 10: Val Accuracy = 0.6429


In [33]:
import pandas as pd

# Загрузка CSV
df = pd.read_csv('sinusitis/full_labels.csv')

# Отбор только снимков с опухолью (где хотя бы один label = 1)
df_with_tumor = df[(df['label1'] == 1) | (df['label2'] == 1)]

# Классификация по стороне опухоли
def classify_side(row):
    if row['label1'] == 1 and row['label2'] == 0:
        return 'Left'
    elif row['label1'] == 0 and row['label2'] == 1:
        return 'Right'
    elif row['label1'] == 1 and row['label2'] == 1:
        return 'Both'
    else:
        return 'None'  # На всякий случай

df_with_tumor['tumor_side'] = df_with_tumor.apply(classify_side, axis=1)

# Подсчёт количества по классам
print("Баланс классов по стороне опухоли:")
print(df_with_tumor['tumor_side'].value_counts())


Баланс классов по стороне опухоли:
tumor_side
Both     37
Left     32
Right     5
Name: count, dtype: int64


/var/folders/rp/j49jf4lj2b36n_gt75n_v8mh0000gn/T/ipykernel_68898/3275257236.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_tumor['tumor_side'] = df_with_tumor.apply(classify_side, axis=1)


In [35]:
import os
import torch
from torchvision import transforms
from PIL import Image
import pandas as pd
from sklearn.metrics import confusion_matrix

# Пути
IMG_DIR = "sinusitis/full_imgs"
LABEL_PATH = "sinusitis/full_labels.csv"

import torch
import torch.nn as nn
from torchvision import models

# Модель для классификации опухоли (есть/нет)
model_tumor = models.resnet18(pretrained=False)
model_tumor.fc = nn.Linear(model_tumor.fc.in_features, 2)
model_tumor.load_state_dict(torch.load("resnet18_fold5.pth"))
model_tumor.eval()

# Модель для определения стороны (Left / Right / Both)
model_side = models.resnet18(pretrained=False)
model_side.fc = nn.Linear(model_side.fc.in_features, 3)
model_side.load_state_dict(torch.load("models_tumor_side/best_model_fold2.pt"))
model_side.eval()


# Предобработка изображений
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Загрузка истинных меток
df_labels = pd.read_csv(LABEL_PATH).set_index("filename")

# Результаты
results = []

# Проход по всем изображениям
with torch.no_grad():
    for filename in os.listdir(IMG_DIR):
        if not filename.endswith(".jpg"):
            continue

        img_path = os.path.join(IMG_DIR, filename)
        image = Image.open(img_path).convert("RGB")
        image_tensor = transform(image).unsqueeze(0)

        # Предсказание наличия опухоли
        output_tumor = model_tumor(image_tensor)
        pred_tumor = int(output_tumor.argmax(1))

        # Предсказание стороны (если есть опухоль)
        if pred_tumor == 1:
            output_side = model_side(image_tensor)
            pred_side = int(output_side.argmax(1))
        else:
            pred_side = None  # нет опухоли — нет стороны

        # Истинные значения
        true_label = df_labels.loc[filename]
        true_tumor = int(true_label["result_label"])
        true_side = int(true_label["label2"]) if true_tumor == 1 else None

        results.append({
            "filename": filename,
            "true_tumor": true_tumor,
            "pred_tumor": pred_tumor,
            "true_side": true_side,
            "pred_side": pred_side,
        })

# Анализ
df = pd.DataFrame(results)

# Метрика по опухоли
print("\n=== Tumor Detection ===")
cm_tumor = confusion_matrix(df["true_tumor"], df["pred_tumor"])
print("Confusion Matrix:\n", cm_tumor)
tn, fp, fn, tp = cm_tumor.ravel()
print(f"False Negative (пропущена опухоль): {fn}")
print(f"False Positive (ошибочно определена опухоль): {fp}")
print(f"Accuracy: {(tp + tn) / (tp + tn + fp + fn):.3f}")

# Метрика по стороне (только если опухоль есть и у модели, и в истине)
side_df = df[(df["true_tumor"] == 1) & (df["pred_tumor"] == 1)]
if not side_df.empty:
    print("\n=== Tumor Side Classification (among tumor cases) ===")
    cm_side = confusion_matrix(side_df["true_side"], side_df["pred_side"])
    print("Confusion Matrix:\n", cm_side)


/Users/arnellatolegen/Desktop/sinus_ct_project/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/arnellatolegen/Desktop/sinus_ct_project/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



=== Tumor Detection ===
Confusion Matrix:
 [[62  0]
 [ 3 71]]
False Negative (пропущена опухоль): 3
False Positive (ошибочно определена опухоль): 0
Accuracy: 0.978

=== Tumor Side Classification (among tumor cases) ===
Confusion Matrix:
 [[29  0  1]
 [ 1  6 34]
 [ 0  0  0]]
